# Responsible AI 
This notebook illustrates input and output guardrails as well as the AI safety mechanisms implemented in the base LLM. As an example, we will assume a simple text to image scneario. For input guardrails, we implement Bedrock Guardrails. If the input prompt appears to pass, we will forward along to the image generating model. In this example, we will use Titan Image Generator. Once the image is generated, we will further run the output through the Claude3 Haiku multi-modal LLM to ensure an additonal layer of safety is implemented before sending the resulting image back to the user. 

### Send in a text prompt to attempt to generate an Image


Install Dependencies

In [1]:
# Install latest version of boto3
# !pip install boto3 s3transfer termcolor ipython --quiet
# !pip install matplotlib  --quiet
import boto3
import pprint

In [2]:
# !pip install --upgrade --force-reinstall boto3 botocore
print(boto3.__version__)

1.34.95


In [7]:
# Built-in libraries
import base64
import io
import json
import os
import sys

# External dependencies
from PIL import Image
import botocore

bedrock_rt = boto3.client('bedrock-runtime')
bedrock_client = boto3.client('bedrock')


Create the Bedrock Guardrails

In [4]:
guardrail_config = {
    'name': 'no-warfare-2',
    'description': 'disallow input prompts for generating images related to military or warfare',
    'topicPolicyConfig': {
        'topicsConfig': [
            {
                'name': 'deny military or warfare related prompts',
                'definition': 'Asking for generating images of mititary equipment like tanks, guns, mitilary drones, bullets, bombs, bunkers, shields etc are not allowed.',
                'examples': [
                    'Generate an image depicting a military tank manufacturing plant',
                    'Image showing bullets being inserted into a gun',
                    'Image showing a mitiary bunker or base',
                    'military drones',
                ],
                'type': 'DENY'
            }
        ]
    },
    'wordPolicyConfig': {
        'wordsConfig': [
            {
                'text': 'tank'
            },
            {
                'text': 'bullet'
            },
            {
                'text': 'drone'
            },
            {
                'text': 'fighter aircrafts'
            },
            {
                'text': 'guns'
            }
        ],
    },
    'blockedInputMessaging': 'I apologize, but I am not able to generate images related to the military. Please try again with another topic',
    'blockedOutputsMessaging': 'I apologize, but I am not able to generate images related to the military. Please try again with another topic',
}

In [ ]:
# create_response = bedrock_client.create_guardrail(**guardrail_config)
# pprint.pprint(create_response)

In [ ]:
# Let's now get all of our updates 
# get_response = bedrock_client.get_guardrail(
#     guardrailIdentifier=create_response['guardrailId'],
#     guardrailVersion='DRAFT'
# )
# pprint.pprint(get_response)

In [ ]:
# To list the DRAFT version of all your guardrails, don’t specify the guardrailIdentifier field. To list all versions of a guardrail, specify the ARN of the guardrail in the guardrailIdentifier field.
# list_guardrails_response = bedrock_client.list_guardrails(
#     guardrailIdentifier=create_response['guardrailArn'],
#     maxResults=5)

# pprint.pprint(list_guardrails_response)

In [38]:
prompt = "a beautiful lake surrounded by trees with a mountain range at the distance"
# prompt = "military drones"
negative_prompts = "poorly rendered, poor background details, poorly drawn mountains, disfigured mountain features"

### Run the text promt through the input guardrails

The Amazon Bedrock `InvokeModel` provides access to Amazon Titan Image Generator by setting the right model ID, and returns a JSON response including a [Base64 encoded string](https://en.wikipedia.org/wiki/Base64) that represents the (PNG) image.

When making an `InvokeModel` request, we need to fill the `body` field with a JSON object that varies depending on the task (`taskType`) you wish to perform viz. text to image, image variation, inpainting or outpainting. The Amazon Titan models supports the following parameters:
* `cfgscale` - determines how much the final image reflects the prompt
* `seed` - a number used to initialize the generation, using the same seed with the same prompt + settings combination will produce the same results
* `numberOfImages` - the number of times the image is sampled and produced
* `quality` - determines the output image quality (`standard` or `premium`)

> ☝️ For more information on available input parameters for the model, refer to the [Amazon Bedrock User Guide](https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-titan-image.html#model-parameters-titan-img-request-body) (Inference parameters > Amazon Titan image models > Model invocation request body fields).

The cell below invokes the Amazon Titan Image Generator model through Amazon Bedrock to create an initial image:

### If safe, genreate the image

In [39]:
# Create payload
body = json.dumps(
    {
        "taskType": "TEXT_IMAGE",
        "textToImageParams": {
            "text": prompt,                    # Required
            "negativeText": negative_prompts   # Optional
        },
        "imageGenerationConfig": {
            "numberOfImages": 1,   # Range: 1 to 5 
            "quality": "standard",  # Options: standard or premium
            "height": 1024,        # Supported height list in the docs 
            "width": 1024,         # Supported width list in the docs
            "cfgScale": 7.5,       # Range: 1.0 (exclusive) to 10.0
            "seed": 42             # Range: 0 to 214783647
        }
    }
)

# Make model request
response = bedrock_rt.invoke_model(
    body=body,
    modelId="amazon.titan-image-generator-v1",
    accept="application/json", 
    contentType="application/json"
)

# Process the image
response_body = json.loads(response.get("body").read())
img1_b64 = response_body["images"][0]

# Debug
print(f"Output: {img1_b64[0:80]}...")

Output: iVBORw0KGgoAAAANSUhEUgAABAAAAAQACAIAAADwf7zUAAEAAElEQVR4nFT9S7Jk27IrhgFwn7H5XkWm...


In [40]:
# Some utility functions

def save_base64_image(img_base64, filename, directory="data/titan"):
    """
    Saves a base64 encoded image to the specified directory.

    Args:
        img_base64 (str): Base64 encoded image data.
        filename (str): Filename to save the image as.
        directory (str, optional): Directory to save the image in. Defaults to "data/titan".

    Returns:
        PIL.Image.Image: The saved image object.
    """
    # Create the directory if it doesn't exist
    os.makedirs(directory, exist_ok=True)

    # Decode and save the image
    img_bytes = base64.decodebytes(bytes(img_base64, "utf-8"))
    img = Image.open(io.BytesIO(img_bytes))
    img.save(os.path.join(directory, filename))

    return img

def get_base64_image(image_path):
    """
    Generates the base64 encoded string representation of an image file.

    Args:
        image_path (str): Path to the image file.

    Returns:
        str: Base64 encoded string representation of the image.
    """
    with open(image_path, "rb") as image_file:
        encoded_string = base64.b64encode(image_file.read())

    return encoded_string.decode("utf-8")

def resize_and_save_base64(input_data, max_size, quality=90, is_base64=False):
    """
    Resizes an image to fit within the specified maximum dimensions and saves it as a base64 encoded string.

    Args:
        input_data (str or PIL.Image.Image): The image data as a base64 string or a PIL Image object.
        max_size (tuple): The maximum width and height (max_width, max_height).
        quality (int, optional): The quality of the saved image (0-100). Defaults to 90.
        is_base64 (bool, optional): Whether the input_data is a base64 string or an image path. Defaults to False.

    Returns:
        str: Base64 encoded string representation of the resized image.
    """
    # Convert input data to a PIL Image object
    if is_base64:
        image_bytes = base64.decodebytes(bytes(input_data, "utf-8"))
        image = Image.open(io.BytesIO(image_bytes))
    else:
        image = Image.open(input_data)

    # Resize the image if necessary
    if image.size[0] > max_size[0] or image.size[1] > max_size[1]:
        image.thumbnail(max_size, Image.Resampling.LANCZOS)

    # Save the resized image to a BytesIO object
    image_data = io.BytesIO()
    image.save(image_data, format='PNG', optimize=True, quality=quality)

    # Encode the image data as a base64 string
    image_data.seek(0)
    base64_data = base64.b64encode(image_data.getvalue()).decode('utf-8')

    return base64_data


In [35]:
filename = "image2.png"
directory="data/titan"

In [36]:
# Debug
print(f"Output: {img1_b64[0:80]}...")


Output: iVBORw0KGgoAAAANSUhEUgAABAAAAAQACAIAAADwf7zUAAEAAElEQVR4nEz9vbJlzbIrAEnKGrP3DRwi...


In [ ]:
filename = "image2.png"
directory="data/titan"

save_base64_image(img1_b64, filename, directory)

In [ ]:
# Base64 encoded image
# from fileinput import filename
base64_image_data = img1_b64
max_size = (1024, 1024)
resized_base64 = resize_and_save_base64(base64_image_data, max_size, is_base64=True)

save_base64_image(resized_base64, filename, directory)

# Image path
# image_path = os.path.join(directory, filename)
# max_size = (1024, 1024)
# resized_base64 = resize_and_save_base64(image_path, max_size)

In [15]:
# image_path = os.path.join(directory, filename)
# img1_b64_temp2 = get_base64_image(image_path)


In [44]:
# Debug
print(f"Output: {img1_b64[0:80]}...")


Output: iVBORw0KGgoAAAANSUhEUgAABAAAAAQACAIAAADwf7zUAAEAAElEQVR4nFT9S7Jk27IrhgFwn7H5XkWm...


### Run the generated image through multi-modal model to check for safety

In [45]:
import base64, re,json
import boto3
from termcolor import colored
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib import rcParams
from IPython.display import display, Image
from PIL import Image
import io
import base64

```python
messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                    "source": {
                        "type": "base64",
                        "media_type": image1_media_type,
                        "data": image1_data,
                    },
                },
                {
                    "type": "text",
                    "text": "Describe this image."
                }
            ],
        }
    ],
```



In [46]:
def generate_vision_answer(bedrock_rt:boto3.client,messages:list, model_id:str, claude_config:dict,system_prompt:str):
    """
    Generates a vision answer using the specified model and configuration.
    
    Parameters:
    - bedrock_rt (boto3.client): The Bedrock runtime client.
    - messages (list): A list of messages.
    - model_id (str): The ID of the model to use.
    - claude_config (dict): The configuration for Claude.
    - system_prompt (str): The system prompt.
    
    Returns:
    - str: The formatted response.
    """
    
    body={'messages': [messages],**claude_config, "system": system_prompt}
    
    response = bedrock_rt.invoke_model(modelId=model_id, body=json.dumps(body))   
    response = json.loads(response['body'].read().decode('utf-8'))
    formated_response= response['content'][0]['text']
    
    return formated_response

In [47]:
%%time

# Model id and claude config
# from https://docs.aws.amazon.com/bedrock/latest/userguide/model-ids.html#model-ids-arns
model_id = "anthropic.claude-3-sonnet-20240229-v1:0"
claude_config = {
    'max_tokens': 6000, 
    'temperature': 0, 
    'anthropic_version': '',  
    'top_p': 1, 
    'stop_sequences': ['Human:']
}

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 6.91 µs


In [48]:
encoded_pngs = []

# we can use the base64 version of the image directly since Titan generated it for us
# Else use the utility function provided above to encode the image
encoded_pngs.append(img1_b64)



In [49]:
content = [{"type": "image", "source": {"type": "base64", "media_type": "image/png", "data": encoded_png}} for encoded_png in encoded_pngs[:20]]
safety_prompt = "\n<guardrail_config>" + json.dumps(guardrail_config) + "</guardrail_config>" + "\n\nAs per the guardrail config provided, the input image is"
content.append({"type": "text", "text": safety_prompt})

# Create message with the prompt and the base64 encoded image
messages={"role": "user", "content": content }
system_prompt=" You are a safety guard model. You will expect in a guardrail configuration json enclosed by <guardrail_config> tags. Reply UNSAFE if the image has any items denied in the guardrail_config. Else reply SAFE" 


# Generate answer
answer= generate_vision_answer(bedrock_rt, messages, model_id, claude_config, system_prompt)
print(colored(answer, "green"))

SAFE

The image depicts a beautiful natural landscape with a lake reflecting the snow-capped mountains in the background. There are no military or warfare-related elements present in the scene. The image shows trees, a body of water, and mountainous terrain, which are all allowed based on the provided guardrail configuration.


In [50]:
# Sanity Check
pprint.pprint(safety_prompt)

('\n'
 '<guardrail_config>{"name": "no-warfare-2", "description": "disallow input '
 'prompts for generating images related to military or warfare", '
 '"topicPolicyConfig": {"topicsConfig": [{"name": "deny military or warfare '
 'related prompts", "definition": "Asking for generating images of mititary '
 'equipment like tanks, guns, mitilary drones, bullets, bombs, bunkers, '
 'shields etc are not allowed.", "examples": ["Generate an image depicting a '
 'military tank manufacturing plant", "Image showing bullets being inserted '
 'into a gun", "Image showing a mitiary bunker or base", "military drones"], '
 '"type": "DENY"}]}, "wordPolicyConfig": {"wordsConfig": [{"text": "tank"}, '
 '{"text": "bullet"}, {"text": "drone"}, {"text": "fighter aircrafts"}, '
 '{"text": "guns"}]}, "blockedInputMessaging": "I apologize, but I am not able '
 'to generate images related to the military. Please try again with another '
 'topic", "blockedOutputsMessaging": "I apologize, but I am not able to 